# Optimizing Python Code with [Cython](https://cython.org/)

In [ ]:
import Cython
from random import random
import math
import numpy as np
import matplotlib.pyplot as plt

# This is needed to load the Cython magic
%load_ext Cython 

## Calculating $\pi$ using Monte Carlo Integration

### Pure Python

In [ ]:
def pi_mc(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

### Cython

#### 1. Use `cython` cell magic without making any code change

In [ ]:
%%cython

from random import random

def pi_mc(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

#### 2. Static type declarations in Cython with `cdef`

Static type declarations allow Cython to step out of the dynamic nature of the Python code and produce efficient C code. The following table summarizes the primitive Cython types.

In [ ]:
%%cython

from random import random

def pi_mc(n=1000):
    '''Calculate PI using Monte Carlo method'''
    cdef int in_circle = 0
    cdef int i
    cdef double x, y
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

#### 3. Using Cython `annotate` option and inspect the generated C code

In [ ]:
%%cython --annotate

from random import random

def pi_mc(int n=1000):
    '''Calculate PI using Monte Carlo method'''
    cdef:
        int in_circle = 0
        int i
        double x, y
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)